In [1]:
# Dependencies and Setup

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import time
import scipy.stats as st
from scipy.stats import linregress

In [30]:
# Output File (CSV)
pyre_data_path = "Resources/California_Fire_Incidents.csv"

In [31]:
pyre_data = pd.read_csv(pyre_data_path)
pyre_data.head()

,AcresBurned,Active,AdminUnit,AirTankers,ArchiveYear,CalFireIncident,CanonicalUrl,ConditionStatement,ControlStatement,Counties,...,Started,Time.1,Status,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,Updated,WaterTenders
0,257314.0,False,Stanislaus National Forest/Yosemite National Park,NaN,2013,True,/incidents/2013/8/17/rim-fire/,NaN,NaN,Tuolumne,...,8/17/13,15:25:00Z,Finalized,NaN,NaN,NaN,NaN,5fb18d4d-213f-4d83-a179-daaf11939e78,2013-09-06T18:30:00Z,NaN
1,30274.0,False,USFS Angeles National Forest/Los Angeles Count...,NaN,2013,True,/incidents/2013/5/30/powerhouse-fire/,NaN,NaN,Los Angeles,...,5/30/13,15:28:00Z,Finalized,NaN,NaN,NaN,NaN,bf37805e-1cc2-4208-9972-753e47874c87,2013-06-08T18:30:00Z,NaN
2,27531.0,False,CAL FIRE Riverside Unit / San Bernardino Natio...,NaN,2013,True,/incidents/2013/7/15/mountain-fire/,NaN,NaN,Riverside,...,7/15/13,13:43:00Z,Finalized,NaN,NaN,NaN,NaN,a3149fec-4d48-427c-8b2c-59e8b79d59db,2013-07-30T18:00:00Z,NaN
3,27440.0,False,Tahoe National Forest,NaN,2013,False,/incidents/2013/8/10/american-fire/,NaN,NaN,Placer,...,8/10/13,16:30:00Z,Finalized,NaN,NaN,NaN,NaN,8213f5c7-34fa-403b-a4bc-da2ace6e6625,2013-08-30T08:00:00Z,NaN
4,24251.0,False,Ventura County Fire/CAL FIRE,NaN,2013,True,/incidents/2013/5/2/springs-fire/,Acreage has been reduced based upon more accur...,NaN,Ventura,...,5/2/13,07:01:00Z,Finalized,6.0,10.0,NaN,NaN,46731fb8-3350-4920-bdf7-910ac0eb715c,2013-05-11T06:30:00Z,11.0


In [32]:
pyre_df = pyre_data[["AcresBurned", "AirTankers", "ArchiveYear", "Counties", "CountyIds","CrewsInvolved", "Dozers", "Engines", "Extinguished", "Days Burned", "Fatalities", "Helicopters", "Injuries", "Latitude", "Location", "MajorIncident", "Name", "PersonnelInvolved", "Started", "StructuresDamaged", "StructuresDestroyed", "StructuresEvacuated", "StructuresThreatened", "UniqueId", "WaterTenders" 
]]
pyre_df.head()

,AcresBurned,AirTankers,ArchiveYear,Counties,CountyIds,CrewsInvolved,Dozers,Engines,Extinguished,Days Burned,...,MajorIncident,Name,PersonnelInvolved,Started,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,WaterTenders
0,257314.0,NaN,2013,Tuolumne,55,NaN,NaN,NaN,9/6/13,20,...,False,Rim Fire,NaN,8/17/13,NaN,NaN,NaN,NaN,5fb18d4d-213f-4d83-a179-daaf11939e78,NaN
1,30274.0,NaN,2013,Los Angeles,19,NaN,NaN,NaN,6/8/13,9,...,False,Powerhouse Fire,NaN,5/30/13,NaN,NaN,NaN,NaN,bf37805e-1cc2-4208-9972-753e47874c87,NaN
2,27531.0,NaN,2013,Riverside,33,NaN,NaN,NaN,7/30/13,15,...,False,Mountain Fire,NaN,7/15/13,NaN,NaN,NaN,NaN,a3149fec-4d48-427c-8b2c-59e8b79d59db,NaN
3,27440.0,NaN,2013,Placer,31,NaN,NaN,NaN,8/30/13,20,...,False,American Fire,NaN,8/10/13,NaN,NaN,NaN,NaN,8213f5c7-34fa-403b-a4bc-da2ace6e6625,NaN
4,24251.0,NaN,2013,Ventura,56,47.0,8.0,117.0,5/11/13,9,...,True,Springs Fire,2167.0,5/2/13,6.0,10.0,NaN,NaN,46731fb8-3350-4920-bdf7-910ac0eb715c,11.0


In [33]:
counties = pyre_df["Counties"].value_counts()
counties

Riverside          146
San Diego           89
Butte               66
Shasta              64
San Luis Obispo     64
Kern                62
Fresno              57
Siskiyou            57
San Bernardino      53
Tehama              51
Lake                49
Los Angeles         46
Monterey            45
Santa Clara         39
El Dorado           37
Madera              36
Lassen              36
Tulare              35
Mariposa            35
Alameda             32
Modoc               31
Ventura             30
Santa Barbara       29
Mendocino           28
Contra Costa        27
Humboldt            25
Napa                25
Calaveras           22
Tuolumne            22
Trinity             21
San Benito          20
Stanislaus          20
Solano              19
Sonoma              18
Placer              17
Nevada              17
Merced              16
Yuba                14
Amador              13
Inyo                12
Yolo                12
Plumas              11
Sacramento          11
Mono       

In [34]:
unique_id = pyre_df["UniqueId"].value_counts()
unique_id.head(30)

5ce45b6c-81b5-422c-9078-8c68dd872c8a    4
90b0daf7-5d84-42d9-bb35-bb78d4faf950    4
af30fe23-6cfd-4fd6-b2d9-91648eeae814    3
676946dc-1b38-4a09-9bbe-9ce638cfed52    2
94a7ae52-fed9-4997-b972-1761aa4b89ca    2
82c2aafa-ef04-49f0-a172-0dfa8a15582e    2
4d07ccc3-32be-4a1d-acac-18333884c97d    2
2437bb57-1d09-4892-8c26-a60661dae1d9    2
a21faf4f-3628-4e24-ac64-8471d9d12560    2
808c34bb-4e64-4eb5-9c3e-2a1e58195074    2
b604ffa7-8cee-4724-bb82-f1f4553a192f    2
ba818380-4001-46a2-ac04-d5f322cdf3b9    2
23cbd274-ded3-43ea-9174-b5615eac3244    2
7ed61f45-7e2b-449b-8ead-d34b75a5e818    2
97ff14e8-6d08-4214-ad3b-847fa3a0e84a    2
d28bc34e-73a8-454d-9e55-dea7bdd40bee    2
be79d28c-767d-4a0d-b168-e86a5842004f    2
4630e9ce-88ec-4159-9bc9-25ff5f8216cc    2
240b1f55-0f4e-448d-a4ce-3ecfec7a6c9d    2
e0d038e9-0926-4c05-8317-ec7075a04b12    2
207f72f7-1453-40dc-919e-3551f7041b22    2
92d7aeea-7abf-47a4-bf76-a4fe0fac65c6    2
487a3f42-fd84-4598-bc48-130ad241c883    1
09f55a45-f261-492f-b496-8744d878ef

In [35]:
pyre_df = pyre_df.set_index('Name')
pyre_df.head()

,AcresBurned,AirTankers,ArchiveYear,Counties,CountyIds,CrewsInvolved,Dozers,Engines,Extinguished,Days Burned,...,Location,MajorIncident,PersonnelInvolved,Started,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,WaterTenders
Name,,,,,,,,,,,,,,,,,,,,,
Rim Fire,257314.0,NaN,2013,Tuolumne,55,NaN,NaN,NaN,9/6/13,20,...,3 miles east of Groveland along Hwy 120,False,NaN,8/17/13,NaN,NaN,NaN,NaN,5fb18d4d-213f-4d83-a179-daaf11939e78,NaN
Powerhouse Fire,30274.0,NaN,2013,Los Angeles,19,NaN,NaN,NaN,6/8/13,9,...,Angeles National Forest,False,NaN,5/30/13,NaN,NaN,NaN,NaN,bf37805e-1cc2-4208-9972-753e47874c87,NaN
Mountain Fire,27531.0,NaN,2013,Riverside,33,NaN,NaN,NaN,7/30/13,15,...,Hwy 243 & Hwy 74 near Mountain Center,False,NaN,7/15/13,NaN,NaN,NaN,NaN,a3149fec-4d48-427c-8b2c-59e8b79d59db,NaN
American Fire,27440.0,NaN,2013,Placer,31,NaN,NaN,NaN,8/30/13,20,...,"Deadwood Ridge, northeast of Foresthill",False,NaN,8/10/13,NaN,NaN,NaN,NaN,8213f5c7-34fa-403b-a4bc-da2ace6e6625,NaN
Springs Fire,24251.0,NaN,2013,Ventura,56,47.0,8.0,117.0,5/11/13,9,...,Southbound Highway 101 at Camarillo Springs Ro...,True,2167.0,5/2/13,6.0,10.0,NaN,NaN,46731fb8-3350-4920-bdf7-910ac0eb715c,11.0


In [36]:
unique_id2 = pyre_df.groupby(["UniqueId"]).count()
unique_id2

,AcresBurned,AirTankers,ArchiveYear,Counties,CountyIds,CrewsInvolved,Dozers,Engines,Extinguished,Days Burned,...,Latitude,Location,MajorIncident,PersonnelInvolved,Started,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,WaterTenders
UniqueId,,,,,,,,,,,,,,,,,,,,,
00089805-cdee-4607-8b67-1653bee463bc,1,0,1,1,1,1,0,1,1,1,...,1,1,1,1,1,0,0,0,0,1
0011ce0f-5493-4bef-b6e1-4674a0393b0c,1,0,1,1,1,0,0,0,1,1,...,1,1,1,0,1,0,0,0,0,0
0089929d-0b52-46fe-8982-b096b7616f30,1,0,1,1,1,0,0,0,1,1,...,1,1,1,0,1,0,0,0,0,0
0089d3c4-c554-46e1-8eb9-af95e4dfae0c,1,0,1,1,1,0,0,0,1,1,...,1,1,1,0,1,0,0,0,0,0
00a13d84-415e-4106-aa31-2e664ed9eabf,1,0,1,1,1,0,0,0,1,1,...,1,1,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ff8701f0-a12a-4b40-baa9-b58c4e2e31f3,1,0,1,1,1,0,0,0,1,1,...,1,1,1,0,1,0,0,0,0,0
ffb3293b-3349-49c5-a7cb-c488391e8a72,1,0,1,1,1,0,0,0,1,1,...,1,1,1,0,1,0,0,0,0,0
ffb57bcc-80f7-4299-bfcf-c9be3b66d5ff,1,0,1,1,1,0,0,0,1,1,...,1,1,1,0,1,0,0,0,0,0


In [37]:
major = pyre_df.loc[pyre_df["MajorIncident"]==True]
major.head()

major["MajorIncident"].count()

383

In [38]:
major.count()

AcresBurned             383
AirTankers               26
ArchiveYear             383
Counties                383
CountyIds               383
CrewsInvolved           167
Dozers                  119
Engines                 185
Extinguished            378
Days Burned             383
Fatalities               21
Helicopters              83
Injuries                119
Latitude                383
Location                383
MajorIncident           383
PersonnelInvolved       191
Started                 383
StructuresDamaged        66
StructuresDestroyed     160
StructuresEvacuated       0
StructuresThreatened     30
UniqueId                383
WaterTenders            141
dtype: int64

In [39]:
years = pyre_df.groupby(["ArchiveYear"]).count()["UniqueId"]
years

ArchiveYear
2013    162
2014    117
2015    152
2016    185
2017    438
2018    316
2019    266
Name: UniqueId, dtype: int64

In [40]:
years_major = major.groupby(["ArchiveYear"]).count()["UniqueId"]
years_major

ArchiveYear
2013    45
2014    40
2015    52
2016    50
2017    93
2018    74
2019    29
Name: UniqueId, dtype: int64

In [41]:
sample = pd.DataFrame({"Number of Fires per Year": years})
sample

,Number of Fires per Year
ArchiveYear,
2013,162
2014,117
2015,152
2016,185
2017,438
2018,316
2019,266


In [43]:
years_acres = pyre_df.groupby(["ArchiveYear"]).count()["AcresBurned"]
years_acres

ArchiveYear
2013    162
2014    117
2015    152
2016    185
2017    438
2018    316
2019    263
Name: AcresBurned, dtype: int64

In [44]:
years_major_acres = major.groupby(["ArchiveYear"]).count()["AcresBurned"]
years_major_acres

ArchiveYear
2013    45
2014    40
2015    52
2016    50
2017    93
2018    74
2019    29
Name: AcresBurned, dtype: int64

In [45]:
resources_crew = pyre_df.groupby(["ArchiveYear"]).count()["CrewsInvolved"]
resources_crew

ArchiveYear
2013    34
2014    23
2015    32
2016    18
2017    22
2018    19
2019    23
Name: CrewsInvolved, dtype: int64

In [47]:
resources_crew_major = major.groupby(["ArchiveYear"]).count()["CrewsInvolved"]

resources_crew_major

ArchiveYear
2013    34
2014    23
2015    32
2016    18
2017    22
2018    19
2019    19
Name: CrewsInvolved, dtype: int64

In [48]:
resources_personnel = pyre_df.groupby(["ArchiveYear"]).count()["PersonnelInvolved"]
resources_personnel

ArchiveYear
2013    35
2014    28
2015    37
2016    23
2017    27
2018    18
2019    36
Name: PersonnelInvolved, dtype: int64

In [49]:
resources_personnel_major = major.groupby(["ArchiveYear"]).count()["PersonnelInvolved"]
resources_personnel_major

ArchiveYear
2013    35
2014    28
2015    37
2016    23
2017    27
2018    18
2019    23
Name: PersonnelInvolved, dtype: int64

In [51]:
resources_dozers = pyre_df.groupby(["ArchiveYear"]).count()["Dozers"]
resources_dozers

ArchiveYear
2013    30
2014    10
2015    26
2016    13
2017    12
2018    15
2019    17
Name: Dozers, dtype: int64

In [52]:
resources_dozer_major = major.groupby(["ArchiveYear"]).count()["Dozers"]
resources_dozer_major

ArchiveYear
2013    30
2014    10
2015    26
2016    13
2017    12
2018    15
2019    13
Name: Dozers, dtype: int64

In [53]:
resources_engines = pyre_df.groupby(["ArchiveYear"]).count()["Engines"]
resources_engines

ArchiveYear
2013    37
2014    24
2015    38
2016    22
2017    23
2018    21
2019    26
Name: Engines, dtype: int64

In [55]:
resources_engines_major = major.groupby(["ArchiveYear"]).count()["Engines"]
resources_engines_major

ArchiveYear
2013    37
2014    24
2015    38
2016    22
2017    23
2018    21
2019    20
Name: Engines, dtype: int64

In [56]:
resources_watertenders = pyre_df.groupby(["ArchiveYear"]).count()["WaterTenders"]
resources_watertenders

ArchiveYear
2013    31
2014    15
2015    29
2016    15
2017    17
2018    18
2019    21
Name: WaterTenders, dtype: int64

In [57]:
resources_watertenders_major = major.groupby(["ArchiveYear"]).count()["WaterTenders"]
resources_watertenders_major

ArchiveYear
2013    31
2014    15
2015    29
2016    15
2017    17
2018    18
2019    16
Name: WaterTenders, dtype: int64

In [58]:
major_counties = major["Counties"].value_counts()

major_counties

Butte              28
San Diego          27
Shasta             26
Lake               21
Riverside          18
Napa               16
El Dorado          15
Mariposa           14
Fresno             12
Sonoma             11
San Bernardino     11
San Luis Obispo    11
Tehama             11
Madera             11
Mendocino          10
Calaveras          10
Siskiyou            9
Amador              8
Inyo                7
Ventura             7
Colusa              6
Solano              6
Tuolumne            6
Los Angeles         6
Humboldt            6
Monterey            6
Santa Barbara       5
Yolo                5
Nevada              5
Tulare              5
Placer              4
Trinity             4
Kern                4
San Benito          4
Modoc               3
Lassen              3
Yuba                3
Contra Costa        3
Glenn               3
Alameda             2
Santa Cruz          2
Santa Clara         2
Marin               1
Kings               1
Merced              1
Mono      

In [27]:
# make graph

In [60]:
inc_2019 = pyre_df.loc[pyre_df["ArchiveYear"]==2019]
inc_2019.head()

,AcresBurned,AirTankers,ArchiveYear,Counties,CountyIds,CrewsInvolved,Dozers,Engines,Extinguished,Days Burned,...,Location,MajorIncident,PersonnelInvolved,Started,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,WaterTenders
Name,,,,,,,,,,,,,,,,,,,,,
Easy Fire,1806.0,NaN,2019,Ventura,56,NaN,NaN,NaN,11/2/19,3,...,"West Easy Street and West Los Angeles Avenue, ...",False,NaN,10/30/19,NaN,NaN,NaN,NaN,00e6d6d5-f168-45fa-817a-f7758a5170b7,NaN
Gaines Fire,1300.0,NaN,2019,Mariposa,22,NaN,NaN,NaN,NaN,-43693,...,"Mt Gaines Rd and Bear Valley Rd, Bear Valley",False,NaN,8/16/19,NaN,NaN,NaN,NaN,547b7d19-9b75-4c11-8b88-56adb6460a3c,NaN
Sandalwood Fire,1011.0,NaN,2019,Riverside,33,10.0,NaN,17.0,NaN,-43748,...,Calimesa Boulevard and Sandalwood Drive,True,247.0,10/10/19,16.0,74.0,NaN,NaN,7ff84b11-c3fb-4f5c-8a00-07243d4a4e33,1.0
Rawson Fire,605.0,NaN,2019,Tehama,52,NaN,NaN,NaN,10/28/19,1,...,"Gyle Road and Long Dirt Road, Northwest of Cor...",False,NaN,10/27/19,NaN,NaN,NaN,NaN,828b2492-fc44-4fc0-97fa-f0b439e6fa0d,NaN
Hill Fire,494.0,NaN,2019,Riverside,33,5.0,2.0,14.0,11/2/19,3,...,"Granite Hill and Pyrite Street, in Jurupa Valley",True,126.0,10/30/19,NaN,NaN,NaN,NaN,8f23653c-1a4c-4609-bfb3-72b1665eab69,1.0


In [78]:
major_minor_inc_2019 = inc_2019.groupby(["ArchiveYear", "MajorIncident", "Name"]).count()["CrewsInvolved"]
major_minor_inc_2019 = major_minor_inc_2019.reset_index()
major_minor_inc_2019.loc[(major_minor_inc_2019["CrewsInvolved"]>=1)&(major_minor_inc_2019["MajorIncident"]==False)]

,ArchiveYear,MajorIncident,Name,CrewsInvolved
5,2019,False,American Fire,1
45,2019,False,Creek Fire,1
99,2019,False,Kennedy Fire,1
139,2019,False,Oak Fire,1


In [79]:
major_minor_inc_2019 = inc_2019.groupby(["ArchiveYear", "MajorIncident", "Name"]).count()["Dozers"]
major_minor_inc_2019 = major_minor_inc_2019.reset_index()
major_minor_inc_2019.loc[(major_minor_inc_2019["Dozers"]>=1)&(major_minor_inc_2019["MajorIncident"]==False)]

,ArchiveYear,MajorIncident,Name,Dozers
5,2019,False,American Fire,1
45,2019,False,Creek Fire,1
66,2019,False,Forbestown Fire,1
99,2019,False,Kennedy Fire,1


In [80]:
major_minor_inc_2019 = inc_2019.groupby(["ArchiveYear", "MajorIncident", "Name"]).count()["Engines"]
major_minor_inc_2019 = major_minor_inc_2019.reset_index()
major_minor_inc_2019.loc[(major_minor_inc_2019["Engines"]>=1)&(major_minor_inc_2019["MajorIncident"]==False)]

,ArchiveYear,MajorIncident,Name,Engines
5,2019,False,American Fire,1
45,2019,False,Creek Fire,1
66,2019,False,Forbestown Fire,1
99,2019,False,Kennedy Fire,1
139,2019,False,Oak Fire,1
146,2019,False,Patterson Fire,1


In [82]:
major_minor_inc_2019 = inc_2019.groupby(["ArchiveYear", "MajorIncident", "Name"]).count()["PersonnelInvolved"]
major_minor_inc_2019 = major_minor_inc_2019.reset_index()
major_minor_inc_2019.loc[(major_minor_inc_2019["PersonnelInvolved"]>=1)&(major_minor_inc_2019["MajorIncident"]==False)]

,ArchiveYear,MajorIncident,Name,PersonnelInvolved
5,2019,False,American Fire,1
11,2019,False,Bautista Fire,1
25,2019,False,Broder Fire,1
45,2019,False,Creek Fire,1
66,2019,False,Forbestown Fire,1
99,2019,False,Kennedy Fire,1
104,2019,False,Lime Fire,1
107,2019,False,Lone Fire,1
139,2019,False,Oak Fire,1
146,2019,False,Patterson Fire,1


In [83]:
major_minor_inc_2019 = inc_2019.groupby(["ArchiveYear", "MajorIncident", "Name"]).count()["WaterTenders"]
major_minor_inc_2019 = major_minor_inc_2019.reset_index()
major_minor_inc_2019.loc[(major_minor_inc_2019["WaterTenders"]>=1)&(major_minor_inc_2019["MajorIncident"]==False)]

,ArchiveYear,MajorIncident,Name,WaterTenders
5,2019,False,American Fire,1
45,2019,False,Creek Fire,1
66,2019,False,Forbestown Fire,1
99,2019,False,Kennedy Fire,1
139,2019,False,Oak Fire,1


In [73]:
minor_2019 = inc_2019.loc[inc_2019["MajorIncident"]==False]
minor_2019

,AcresBurned,AirTankers,ArchiveYear,Counties,CountyIds,CrewsInvolved,Dozers,Engines,Extinguished,Days Burned,...,Location,MajorIncident,PersonnelInvolved,Started,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,WaterTenders
Name,,,,,,,,,,,,,,,,,,,,,
Easy Fire,1806.0,NaN,2019,Ventura,56,NaN,NaN,NaN,11/2/19,3,...,"West Easy Street and West Los Angeles Avenue, ...",False,NaN,10/30/19,NaN,NaN,NaN,NaN,00e6d6d5-f168-45fa-817a-f7758a5170b7,NaN
Gaines Fire,1300.0,NaN,2019,Mariposa,22,NaN,NaN,NaN,NaN,-43693,...,"Mt Gaines Rd and Bear Valley Rd, Bear Valley",False,NaN,8/16/19,NaN,NaN,NaN,NaN,547b7d19-9b75-4c11-8b88-56adb6460a3c,NaN
Rawson Fire,605.0,NaN,2019,Tehama,52,NaN,NaN,NaN,10/28/19,1,...,"Gyle Road and Long Dirt Road, Northwest of Cor...",False,NaN,10/27/19,NaN,NaN,NaN,NaN,828b2492-fc44-4fc0-97fa-f0b439e6fa0d,NaN
Foothills Fire,355.0,NaN,2019,Placer,31,NaN,NaN,NaN,NaN,-43794,...,"Foothills and McAnally Drive, northwest of Ros...",False,NaN,11/25/19,NaN,NaN,NaN,NaN,77d64e4c-a50f-4f4d-a750-e6a6f0ddadc9,NaN
Cross Fire,289.0,NaN,2019,Monterey,27;40,NaN,NaN,NaN,10/11/19,2,...,Indian Valley Road and Monterey County Line ne...,False,NaN,10/9/19,NaN,NaN,NaN,NaN,207f72f7-1453-40dc-919e-3551f7041b22,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Eagle Fire,9.0,NaN,2019,Riverside,33,NaN,NaN,NaN,10/10/19,0,...,"Eagle Canyon Rd. and Cajalco Rd., southwest of...",False,NaN,10/10/19,NaN,NaN,NaN,NaN,6e93b252-99a3-4214-9921-238373b17535,NaN
Long Fire,2.0,NaN,2019,Nevada,29,NaN,NaN,NaN,6/28/19,0,...,"Off of Long Point Road and Old Mill Road, Sou...",False,NaN,6/28/19,NaN,NaN,NaN,NaN,b38c0563-b321-431b-9174-6336c5a0d449,NaN
Cashe Fire,NaN,NaN,2019,Yolo,57,NaN,NaN,NaN,NaN,-43794,...,"County Road 102 and County Road 17, North of W...",False,NaN,11/25/19,NaN,NaN,NaN,NaN,9c26f915-1b33-422d-b30a-9eb4da6fd729,NaN


In [29]:
major_inc_2019 = major.loc[major["ArchiveYear"]==2019]
major_inc_2019

,AcresBurned,AirTankers,ArchiveYear,Counties,CountyIds,CrewsInvolved,Dozers,Engines,Extinguished,Days Burned,...,Location,MajorIncident,PersonnelInvolved,Started,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,WaterTenders
Name,,,,,,,,,,,,,,,,,,,,,
Sandalwood Fire,1011.0,NaN,2019,Riverside,33,10.0,NaN,17.0,NaN,-43748,...,Calimesa Boulevard and Sandalwood Drive,True,247.0,10/10/19,16.0,74.0,NaN,NaN,7ff84b11-c3fb-4f5c-8a00-07243d4a4e33,1.0
Hill Fire,494.0,NaN,2019,Riverside,33,5.0,2.0,14.0,11/2/19,3,...,"Granite Hill and Pyrite Street, in Jurupa Valley",True,126.0,10/30/19,NaN,NaN,NaN,NaN,8f23653c-1a4c-4609-bfb3-72b1665eab69,1.0
Miller Fire,37.0,2.0,2019,San Diego,37,4.0,2.0,10.0,10/27/19,2,...,"Miller Lane and Cole Grade Lane, North of Vall...",True,100.0,10/25/19,1.0,3.0,NaN,NaN,7ff229e4-be94-4319-8916-19eb43753843,2.0
Vista Fire,25.0,NaN,2019,San Diego,37,NaN,NaN,NaN,7/18/19,1,...,Off Vista De La Montana in the Dehesa area,True,NaN,7/17/19,NaN,NaN,NaN,NaN,06fa2750-afea-4f88-b1c4-48b9bc21f4eb,NaN
Kincade Fire,77758.0,NaN,2019,Sonoma,49,2.0,0.0,5.0,11/6/19,14,...,"John Kincade Road and Burned Mountain Road, no...",True,55.0,10/23/19,60.0,374.0,NaN,0.0,809d101c-8aa3-4a53-bf20-f38ae7ec7941,2.0
Taboose Fire,10296.0,NaN,2019,Inyo,14,8.0,1.0,8.0,11/21/19,76,...,south west of Big Pine and north west of Aberdeen,True,269.0,9/6/19,NaN,NaN,NaN,NaN,e7ea56c5-4290-4406-b0a5-c9d1718e9700,4.0
Maria Fire,9999.0,NaN,2019,Ventura,56,17.0,NaN,45.0,11/6/19,6,...,"off of Bradley Road and Solano Verde Dr, Top o...",True,626.0,10/31/19,NaN,4.0,NaN,NaN,31852a71-e639-496f-9e7d-0968d1c218b3,3.0
Red Bank Fire,8838.0,3.0,2019,Tehama,52,53.0,27.0,98.0,9/13/19,8,...,"Off Hammer Loop Road and Petty John Road, West...",True,1936.0,9/5/19,NaN,2.0,NaN,NaN,14ad161c-bfc6-41ae-af84-a464084a4142,23.0
Saddle Ridge Fire,8799.0,NaN,2019,Los Angeles,19,NaN,NaN,NaN,10/31/19,21,...,"Saddle Ridge Rd, Sylmar",True,206.0,10/10/19,88.0,19.0,NaN,NaN,c52717d6-0297-4c5b-a0f1-f17abcd7ac27,NaN
